In [1]:
%load_ext autoreload
%autoreload 2

# Pair

### Search for some products from which to build pairs and stacks of pairs

In [2]:
import asf_search as asf

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
reference = results[0]

args = asf.ASFSearchOptions(
    **{"start": '2022-02-10', "end": '2022-07-01'}
)
s = reference.stack(args)

### Create a Pair object from 2 products

In [3]:
pair = asf.Pair(reference, s[2])

print(f"pair.ref.properties['sceneName']: {pair.ref.properties['sceneName']}")
print(f"pair.ref_date: {pair.ref_date}\n")

print(f"pair.sec.properties['sceneName']: {pair.sec.properties['sceneName']}")
print(f"pair.sec_date: {pair.sec_date}\n")

print(f"pair.temporal: {pair.temporal}")
print(f"pair.perpendicular: {pair.perpendicular}")

pair.ref.properties['sceneName']: S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252
pair.ref_date: 2022-02-15

pair.sec.properties['sceneName']: S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E
pair.sec_date: 2022-03-11

pair.temporal: 24 days, 0:00:00
pair.perpendicular: -73


3. Check the estimated coherence of the pair

In [4]:
pair.estimate_mean_coherence()

16.952704858593183

4. Create a new pair with a long temporal baseline

In [5]:
long_pair = pair = asf.Pair(reference, s[10])
print(f"long_pair.temporal: {long_pair.temporal}")

long_pair.temporal: 120 days, 0:00:00


5. Since the temporal baseline is greater than 48, an exception is raised when checking coherence

In [6]:
long_pair.estimate_mean_coherence()

Exception: Coherence dataset includes temporal baselines up to 48 days.
            Temporal baseline: 120 days

3. Test pair equivalence. Two pairs are equal if they both share the same reference and secondary dates.

In [7]:
from copy import deepcopy

a = deepcopy(pair)
b = deepcopy(pair)
c = asf.Pair(reference, s[11])
print(f"a.ref_date: {a.ref_date}, a.sec_date: {a.sec_date}")
print(f"b.ref_date: {b.ref_date}, b.sec_date: {b.sec_date}")
print(f"c.ref_date: {c.ref_date}, c.sec_date: {c.sec_date}")
print(f"a == b: {a == b}")
print(f"a == c: {a == c}")

a.ref_date: 2022-02-15, a.sec_date: 2022-06-15
b.ref_date: 2022-02-15, b.sec_date: 2022-06-15
c.ref_date: 2022-02-15, c.sec_date: 2022-06-27
a == b: True
a == c: False


# Stack

### Create a Stack from the reference scene object instantiated above

Stack accepts ASFSearchOptions, which are used in a stack_from_product search

In [8]:
args = asf.ASFSearchOptions(
    **{"start": '2020-01-01', "end": '2020-03-02'}
)

stack = asf.Stack(reference, args)
stack.full_stack

### Remove pairs from the full stack, identifying them by ref and sec dates

Note that both full_stack and subset_stack contain the same Pair objects, so keeping both stacks does not eat up a ton of extra memory

In [9]:
import pandas as pd
from datetime import datetime, date

stack.remove_pairs([
    (stack.full_stack[0].ref_date, stack.full_stack[0].sec_date),
    (stack.full_stack[1].ref_date, stack.full_stack[1].sec_date),
    (stack.full_stack[2].ref_date, stack.full_stack[2].sec_date)
])

stack.subset_stack

## Network

Create a Network from the same reference scene. 

Attempts to create a connected, seasonal SBAS stack based on:
- georeference scene
- season
- start date
- end date
- perpendicular baseline
- temporal baseline
  

In [10]:
import pandas as pd
from datetime import datetime

season = ("1-1", "6-25")

def get_julian_season(self) -> tuple[int,int]:
    season_start_ts = pd.Timestamp(
        datetime.strptime(f"{season[0]}-0001", "%m-%d-%Y"), tz="UTC"
        )
    season_start_day = season_start_ts.timetuple().tm_yday
    season_end_ts = pd.Timestamp(
        datetime.strptime(f"{season[1]}-0001", "%m-%d-%Y"), tz="UTC"
    )
    season_end_day = season_end_ts.timetuple().tm_yday
    return (season_start_day, season_end_day)


opts = asf.ASFSearchOptions(
    **{
        "start": '2021-01-01', 
        "end": '2023-03-02', 
        "season": get_julian_season(season)
        }
)

network = asf.Network(
    reference,
    perp_baseline=50, 
    temporal_baseline=36,
    bridge_target_date='3-1',
    opts=opts)
network

In [11]:
print(len(network.full_stack))
print(len(network._date_pair_remove_list))
print(len(network.subset_stack))

561
494
67


In [12]:
a = network.plot()